<a href="https://colab.research.google.com/github/sashkosko/import_export_ee/blob/main/zvit_diagramy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re

# Новий розділ

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
df = df[df['Зона бойових дій'].notna()]

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
# -- Генератори (а розрізі потужності і регіонів)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[(df['Тип потреби'] == 'Електростанції (генератори) та обладнання') | (df['Вид потреби'] == 'Газотурбінні установки')]
# Видалимо рядки де в стовбчику "Вид потреби" значення "Генератори (запчастини, сервісні, ремонтні комплекти, інше)"
df = df[df['Вид потреби'] != 'Генератори (запчастини, сервісні, ремонтні комплекти, інше)']
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Область', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцями
df = df.groupby(['Вид потреби', 'Область']).sum()
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Генератори')

In [ ]:
# -- Генератори (а розрізі потужності)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[(df['Тип потреби'] == 'Електростанції (генератори) та обладнання') | (df['Вид потреби'] == 'Газотурбінні установки')]
# Видалимо рядки де в стовбчику "Вид потреби" значення "Генератори (запчастини, сервісні, ремонтні комплекти, інше)"
df = df[df['Вид потреби'] != 'Генератори (запчастини, сервісні, ремонтні комплекти, інше)']
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем "Вид потреби"
# df = df.groupby(['Вид потреби']).sum()
df = df.groupby('Вид потреби', as_index=False).sum()
# Щоб результат був відсортований в потрібному порядку (генератори за потужністю а потім газотурбінні установки), напишемо такий словник
names = {"<10 кВт": 1, "10-30 кВт": 2, "30-100 кВт": 3, "100-300 кВт": 4, "300-500 кВт": 5, "500-700 кВт": 6, "700-1000 кВт": 7, "1000-1500 кВт": 8, "1500-2000 кВт": 9, ">2000 кВт": 10, "Газотурбінні установки": 11}
# Додамо стовбчик num
df['number'] = df['Вид потреби'].map(names)
df = df.sort_values(by='number')
# Залишимо потрібні стовбчики
df = df[['Вид потреби', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]

# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Генератори1')

In [ ]:
# -- Генератори (а розрізі областей без розбивання на типи потужності)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[(df['Тип потреби'] == 'Електростанції (генератори) та обладнання') | (df['Вид потреби'] == 'Газотурбінні установки')]
# Видалимо рядки де в стовбчику "Вид потреби" значення "Генератори (запчастини, сервісні, ремонтні комплекти, інше)"
df = df[df['Вид потреби'] != 'Генератори (запчастини, сервісні, ремонтні комплекти, інше)']
# Залишимо в df лише такі стовбці
df = df[['Область', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Область']).sum()
# Відсортуємо df по стовбцю
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Генератори0')

In [ ]:
# -- Генератори (ТОП5 компаній з областями)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
# df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[(df['Тип потреби'] == 'Електростанції (генератори) та обладнання') | (df['Вид потреби'] == 'Газотурбінні установки')]
# Видалимо рядки де в стовбчику "Вид потреби" значення "Генератори (запчастини, сервісні, ремонтні комплекти, інше)"
df = df[df['Вид потреби'] != 'Генератори (запчастини, сервісні, ремонтні комплекти, інше)']
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Генератори3')

In [ ]:
# -- Генератори (Це для ТОП5 найменш задоволених потреб)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
# df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[(df['Тип потреби'] == 'Електростанції (генератори) та обладнання') | (df['Вид потреби'] == 'Газотурбінні установки')]
# Видалимо рядки де в стовбчику "Вид потреби" значення "Генератори (запчастини, сервісні, ремонтні комплекти, інше)"
df = df[df['Вид потреби'] != 'Генератори (запчастини, сервісні, ремонтні комплекти, інше)']
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Незабезпечена потреба, абс. знач', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Генератори5')

In [ ]:
# -- Генератори (забезпечено самостійно)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
# df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[(df['Тип потреби'] == 'Електростанції (генератори) та обладнання') | (df['Вид потреби'] == 'Газотурбінні установки')]
# Видалимо рядки де в стовбчику "Вид потреби" значення "Генератори (запчастини, сервісні, ремонтні комплекти, інше)"
df = df[df['Вид потреби'] != 'Генератори (запчастини, сервісні, ремонтні комплекти, інше)']
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг', 'Забезпечено самостійно, гумдопомога', 'Забезпечено самостійно, закупівлі', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Вид потреби']).sum()
# Відсортуємо df по стовбцю
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Генератори4')

In [ ]:
# Генератори (ті що відправлені без потреби, в розрізі потужності)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[(df['Тип потреби'] == 'Електростанції (генератори) та обладнання') | (df['Вид потреби'] == 'Газотурбінні установки')]
# Видалимо рядки де в стовбчику "Вид потреби" значення "Генератори (запчастини, сервісні, ремонтні комплекти, інше)"
df = df[df['Вид потреби'] != 'Генератори (запчастини, сервісні, ремонтні комплекти, інше)']
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Вид потреби']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Генератори_відгрузки')

,Обсяг
Вид потреби,
10-30 кВт,28.0
100-300 кВт,41.0
1000-1500 кВт,3.0
30-100 кВт,123.0
300-500 кВт,11.0
700-1000 кВт,1.0
<10 кВт,213.0


In [ ]:
# Генератори (ті що відправлені без потреби, в розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[(df['Тип потреби'] == 'Електростанції (генератори) та обладнання') | (df['Вид потреби'] == 'Газотурбінні установки')]
# Видалимо рядки де в стовбчику "Вид потреби" значення "Генератори (запчастини, сервісні, ремонтні комплекти, інше)"
df = df[df['Вид потреби'] != 'Генератори (запчастини, сервісні, ремонтні комплекти, інше)']
# Залишимо в df лише такі стовбці
df = df[['Область', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Область']).sum()

# Відсортуємо df по стовбцю
df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Генератори_відгрузки2')

In [ ]:
# Генератори (пріоритетні потреби, за потужністю)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/пріоритетні потреби.xlsx', sheet_name='Лише потрібні категорії', engine='openpyxl')
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[(df['Тип потреби'] == 'Електростанції (генератори) та обладнання') | (df['Вид потреби'] == 'Газотурбінні установки')]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг 1 пріоритет', 'Загальний обсяг 2 пріоритет']]
# Згрупуємо дані за стовбцями
df = df.groupby('Вид потреби', as_index=False).sum()
# Щоб результат був відсортований в потрібному порядку (генератори за потужністю а потім газотурбінні установки), напишемо такий словник
names = {"Генератор <10 кВт": 1, "Генератор 10-30 кВт": 2, "Генератор 30-100 кВт": 3, "Генератор 100-300 кВт": 4, "Генератор 300-500 кВт": 5, "Генератор 500-700 кВт": 6, "Генератор 700-1000 кВт": 7, "Генератор 1000-1500 кВт": 8, "Газотурбінні установки": 9}
# Додамо стовбчик num
df['number'] = df['Вид потреби'].map(names)
df = df.sort_values(by='number')
# Залишимо потрібні стовбчики
df = df[['Вид потреби', 'Загальний обсяг 1 пріоритет', 'Загальний обсяг 2 пріоритет']]
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Пріор_Генератори')

In [ ]:
# -- Задоволення потреб і кількість потреб (всі типи потреб)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише такі стовбці
df = df[['Тип потреби', 'Вид потреби', 'Виконання, %']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Тип потреби', 'Вид потреби']).mean()

# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Common1')

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
<ipython-input-16-e3fb2d34f899>:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['Тип потреби', 'Вид потреби']).mean()


In [ ]:
# -- Оливи трансформаторні (а розрізі категорій)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання"
df = df[df['Вид потреби'] == 'Оливи трансформаторні']
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Вид потреби']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Оливи транс.01')

Exception ignored in: <function ZipFile.__del__ at 0x7fa9d240add0>
Traceback (most recent call last):
  File "/usr/lib/python3.10/zipfile.py", line 1821, in __del__
    self.close()
  File "/usr/lib/python3.10/zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [ ]:
# -- Оливи трансформаторні (а розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання"
df = df[df['Вид потреби'] == 'Оливи трансформаторні']
# Залишимо в df лише такі стовбці
df = df[['Область', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Оливи транс.02')

In [ ]:
# -- Оливи трансформаторні (ТОП5 компаній з позначенням області)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
df = df[df['Вид потреби'] == 'Оливи трансформаторні']
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Оливи транс.03')

In [ ]:
# -- Оливи трансформаторні (ТОП5 невдах з позначенням області)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
df = df[df['Вид потреби'] == 'Оливи трансформаторні']
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Незабезпечена потреба, абс. знач', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Оливи транс.04')

In [ ]:
# Оливи трансформаторні (ті що відправлені без потреби, в розрізі категорії)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[(df['Вид потреби'] == 'Оливи трансформаторні')]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Вид потреби']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Оливи_відгрузки01')

In [ ]:
# Оливи трансформаторні (ті що відправлені без потреби, в розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[(df['Вид потреби'] == 'Оливи трансформаторні')]
# Залишимо в df лише такі стовбці
df = df[['Область', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Область']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Оливи_відгрузки02')

In [ ]:
# Оливи (пріоритетні потреби, за категорією)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/пріоритетні потреби.xlsx', sheet_name='Лише потрібні категорії', engine='openpyxl')
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[(df['Вид потреби'] == 'Оливи трансформаторні')]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг 1 пріоритет', 'Загальний обсяг 2 пріоритет']]
# Згрупуємо дані за стовбцями
df = df.groupby('Вид потреби').sum()
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Пріор_Оливи01')

In [ ]:
# -- Опори ліній електропередач (а розрізі категоій)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні або "Опори залізобетонні", або "Опори металеві", або "Опори інші"
df = df[df['Вид потреби'].isin(['Опори залізобетонні', 'Опори металеві'])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Вид потреби']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Опори ЛЕП0')

In [ ]:
# -- Опори ліній електропередач (а розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні або "Опори залізобетонні", або "Опори металеві", або "Опори інші"
df = df[df['Вид потреби'].isin(['Опори залізобетонні', 'Опори металеві'])]
# Залишимо в df лише такі стовбці
df = df[['Область', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Опори ЛЕП')

In [ ]:
# -- Опори ліній електропередач (ТОП5 з областями)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні або "Опори залізобетонні", або "Опори металеві", або "Опори інші"
df = df[df['Вид потреби'].isin(['Опори залізобетонні', 'Опори металеві'])]
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Опори ЛЕП_1')

In [ ]:
# -- Опори ліній електропередач (ТОП5 невдах з областями)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні або "Опори залізобетонні", або "Опори металеві", або "Опори інші"
df = df[df['Вид потреби'].isin(['Опори залізобетонні', 'Опори металеві'])]
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Незабезпечена потреба, абс. знач', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Опори ЛЕП_3')

In [ ]:
# Опори ЛЕП (ті що відправлені без потреби, в розрізі категорії)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# Залишимо в df лише ті рядки
df = df[df['Вид потреби'].isin(['Опори залізобетонні', 'Опори металеві'])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Вид потреби']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='ЛЕП_відгрузки01')

In [ ]:
# Опори ЛЕП (ті що відправлені без потреби, в розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# Залишимо в df лише ті рядки
df = df[df['Вид потреби'].isin(['Опори залізобетонні', 'Опори металеві'])]
# Залишимо в df лише такі стовбці
df = df[['Область', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Область']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='ЛЕП_відгрузки02')

In [ ]:
# Опори ЛЕП (пріоритетні потреби, за категорією)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/пріоритетні потреби.xlsx', sheet_name='Лише потрібні категорії', engine='openpyxl')
# Залишимо в df лише ті рядки,
df = df[df['Вид потреби'].isin(['Опори залізобетонні', 'Опори металеві'])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг 1 пріоритет', 'Загальний обсяг 2 пріоритет']]
# Згрупуємо дані за стовбцями
df = df.groupby('Вид потреби').sum()
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Пріор_ЛЕП')

In [ ]:
# -- Вимикачі (в розрізі категорій)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(['Вимикачі вакуумні', 'Вимикачі елегазові'])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Вид потреби']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Вимикачі')

In [ ]:
# -- Вимикачі (в розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(['Вимикачі вакуумні', 'Вимикачі елегазові'])]
# Залишимо в df лише такі стовбці
df = df[['Область', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Вимикачі_1')

In [ ]:
# -- Вимикачі (ТОП3 з областями)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(['Вимикачі вакуумні', 'Вимикачі елегазові'])]
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Залишимо лише 3 перших рядків
df = df.head(3)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Вимикачі_2')

In [ ]:
# -- Вимикачі (ТОП3 невдах з областями)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(['Вимикачі вакуумні', 'Вимикачі елегазові'])]
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Незабезпечена потреба, абс. знач', ascending=False)
# Залишимо лише 3 перших рядків
df = df.head(3)
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Вимикачі_3_1')

In [ ]:
# Вимикачі (ті що відправлені без потреби, в розрізі категорії)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# Залишимо в df лише ті рядки
df = df[df['Вид потреби'].isin(['Вимикачі вакуумні', 'Вимикачі елегазові'])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Вид потреби']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Вимикачі_відгрузки01')

In [ ]:
# Вимикачі (ті що відправлені без потреби, в розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# Залишимо в df лише ті рядки
df = df[df['Вид потреби'].isin(['Вимикачі вакуумні', 'Вимикачі елегазові'])]
# Залишимо в df лише такі стовбці
df = df[['Область', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Область']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Вимикачі_відгрузки02')

In [ ]:
# Вимикачі (пріоритетні потреби, за категорією)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/пріоритетні потреби.xlsx', sheet_name='Лише потрібні категорії', engine='openpyxl')
# Залишимо в df лише ті рядки,
df = df[df['Вид потреби'].isin(['Вимикачі вакуумні', 'Вимикачі елегазові'])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг 1 пріоритет', 'Загальний обсяг 2 пріоритет']]
# Згрупуємо дані за стовбцями
df = df.groupby('Вид потреби').sum()
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Пріор_вимикачі')

In [ ]:
# -- Автотранспорт (в розрізі видів транспорту)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
#Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення дорівнює "Автотранспорт та спецтехніка"
df = df[df['Тип потреби'] == 'Автотранспорт та спецтехніка']
# Видалимо рядки
df = df[~df['Повне найменування потреби'].isin(["Шина 26,5-25 E3/L3 28PR TL для Hyundai", "Шина 23.5-25 E3/L3 20 PR TL для Shantui", "Шина 18.00-25 L5S 32PR TL для МТ-2010", "Шина 16.00-25 L5S 32 PR TT для РАUS", "Гальма головного підйому для крана козлового КК 125+10 Konecranes", "Гальма головного підйому для крана козлового КК 420+ 420+16 Konecranes", "Кабіна бурильника для комплектації бурового верстату ZJ50DBS", "Бурова вежа для комплектації бурового верстату ZJ50DBS"])]
df['Підтип потреби'] = 'Спецтехніка'
# Якщо в стовбчику "Вид потреби" значення дорівнює "Автотранспорт (інше)", або "Автомобілі бригадні вантажо-пасажирські", то в стовбчику "Підтип потреби" внесемо значення "Транспортні засоби"
df.loc[(df['Вид потреби'] == 'Автотранспорт (інше)') | (df['Вид потреби'] == 'Автомобілі бригадні вантажо-пасажирські'), 'Підтип потреби'] = 'Транспортні засоби'
# Залишимо в df лише такі стовбці
df = df[['Підтип потреби', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо df по стовбцях
df = df.groupby(['Підтип потреби']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Авто0')

In [ ]:
# -- Автотранспорт (в розрізі підтипів і областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
#Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення дорівнює "Автотранспорт та спецтехніка"
df = df[df['Тип потреби'] == 'Автотранспорт та спецтехніка']


# Видалимо рядки
df = df[~df['Повне найменування потреби'].isin(["Шина 26,5-25 E3/L3 28PR TL для Hyundai", "Шина 23.5-25 E3/L3 20 PR TL для Shantui", "Шина 18.00-25 L5S 32PR TL для МТ-2010", "Шина 16.00-25 L5S 32 PR TT для РАUS", "Гальма головного підйому для крана козлового КК 125+10 Konecranes", "Гальма головного підйому для крана козлового КК 420+ 420+16 Konecranes", "Кабіна бурильника для комплектації бурового верстату ZJ50DBS", "Бурова вежа для комплектації бурового верстату ZJ50DBS"])]

df['Підтип потреби'] = 'Спецтехніка'
# Якщо в стовбчику "Вид потреби" значення дорівнює "Автотранспорт (інше)", або "Автомобілі бригадні вантажо-пасажирські", то в стовбчику "Підтип потреби" внесемо значення "Транспортні засоби"
df.loc[(df['Вид потреби'] == 'Автотранспорт (інше)') | (df['Вид потреби'] == 'Автомобілі бригадні вантажо-пасажирські'), 'Підтип потреби'] = 'Транспортні засоби'
# Залишимо в df лише такі стовбці
df = df[['Підтип потреби', 'Область', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо df по стовбцях
df = df.groupby(['Підтип потреби', 'Область']).sum()
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Авто10')

In [ ]:
# -- Автотранспорт (в розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
#Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення дорівнює "Автотранспорт та спецтехніка"
df = df[df['Тип потреби'] == 'Автотранспорт та спецтехніка']
df['Підтип потреби'] = 'Спецтехніка'
# Якщо в стовбчику "Вид потреби" значення дорівнює "Автотранспорт (інше)", або "Автомобілі бригадні вантажо-пасажирські", то в стовбчику "Підтип потреби" внесемо значення "Транспортні засоби"
df.loc[(df['Вид потреби'] == 'Автотранспорт (інше)') | (df['Вид потреби'] == 'Автомобілі бригадні вантажо-пасажирські'), 'Підтип потреби'] = 'Транспортні засоби'
# Залишимо в df лише такі стовбці
df = df[['Область', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо df по стовбцях
df = df.groupby(['Область']).sum()
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Авто0')

In [ ]:
# -- Автотранспорт (ТОП5 з областями)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
#Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення дорівнює "Автотранспорт та спецтехніка"
df = df[df['Тип потреби'] == 'Автотранспорт та спецтехніка']
df['Підтип потреби'] = 'Спецтехніка'
# Якщо в стовбчику "Вид потреби" значення дорівнює "Автотранспорт (інше)", або "Автомобілі бригадні вантажо-пасажирські", то в стовбчику "Підтип потреби" внесемо значення "Транспортні засоби"
df.loc[(df['Вид потреби'] == 'Автотранспорт (інше)') | (df['Вид потреби'] == 'Автомобілі бригадні вантажо-пасажирські'), 'Підтип потреби'] = 'Транспортні засоби'
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Авто2')

In [ ]:
# -- Автотранспорт (ТОП5 невдах з областями)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
#Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення дорівнює "Автотранспорт та спецтехніка"
df = df[df['Тип потреби'] == 'Автотранспорт та спецтехніка']
df['Підтип потреби'] = 'Спецтехніка'
# Якщо в стовбчику "Вид потреби" значення дорівнює "Автотранспорт (інше)", або "Автомобілі бригадні вантажо-пасажирські", то в стовбчику "Підтип потреби" внесемо значення "Транспортні засоби"
df.loc[(df['Вид потреби'] == 'Автотранспорт (інше)') | (df['Вид потреби'] == 'Автомобілі бригадні вантажо-пасажирські'), 'Підтип потреби'] = 'Транспортні засоби'
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Незабезпечена потреба, абс. знач', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Авто4_1')

In [ ]:
# -- Автотранспорт (В розрізі областей - треба буде вручну вибати відомі компанії і зробити по них таблицю)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
#Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення дорівнює "Автотранспорт та спецтехніка"
df = df[df['Тип потреби'] == 'Автотранспорт та спецтехніка']
df['Підтип потреби'] = 'Спецтехніка'
# Якщо в стовбчику "Вид потреби" значення дорівнює "Автотранспорт (інше)", або "Автомобілі бригадні вантажо-пасажирські", то в стовбчику "Підтип потреби" внесемо значення "Транспортні засоби"
df.loc[(df['Вид потреби'] == 'Автотранспорт (інше)') | (df['Вид потреби'] == 'Автомобілі бригадні вантажо-пасажирські'), 'Підтип потреби'] = 'Транспортні засоби'
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Авто3')

In [ ]:
# Автотранспорт (пріоритетні потреби, за категорією)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/пріоритетні потреби.xlsx', sheet_name='Лише потрібні категорії', engine='openpyxl')
# Залишимо в df лише ті рядки,
df = df[df['Вид потреби'].isin(['Спецтехніка', 'Транспортні засоби'])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг 1 пріоритет', 'Загальний обсяг 2 пріоритет']]
# Згрупуємо дані за стовбцями
df = df.groupby('Вид потреби').sum()
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Пріор_Авто')

In [ ]:
# -- Кабелі (в розрізі видів потреб, треба буде вручну відібрати лише силові кабелі)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
#Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення дорівнює
df = df[df['Тип потреби'] == 'Кабельна продукція']
# Залишимо в df лише ті рядки, де в стовбчику 'Одиниця вимірювання' є 'км', або в стовбчику 'Коефіцієнт од. вимірювання' не порожнє значення
df = df[(df['Одиниця вимірювання'] == 'км') | (df['Коефіцієнт од. вимірювання'].notnull())]
# В df залишимо лише ті рядки де в стовбчику "Вид потреби" значення не рівне "Інша кабельна продукція"
df = df[df['Вид потреби'] != 'Інша кабельна продукція']

# В стовбчику 'Коефіцієнт од. вимірювання' всі порожні значення замінимо на 1
df['Коефіцієнт од. вимірювання'].fillna(1, inplace=True)

# Замінимо порожні значення на нуль в таких стовбчиках
df['Загальний обсяг'].fillna(0, inplace=True)
df['Забезпечено'].fillna(0, inplace=True)
df['Забезпечено самостійно, гумдопомога'].fillna(0, inplace=True)
df['Забезпечено самостійно, закупівлі'].fillna(0, inplace=True)
df['Забезпечено закупівлю за кошти Фонду'].fillna(0, inplace=True)
df['Незабезпечена потреба, абс. знач'].fillna(0, inplace=True)
# Значення в стовбчику "Коефіцієнт од. вимірювання"
df['Загальний обсяг'] = df['Загальний обсяг'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено'] = df['Забезпечено'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено самостійно, гумдопомога'] = df['Забезпечено самостійно, гумдопомога'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено самостійно, закупівлі'] = df['Забезпечено самостійно, закупівлі'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено закупівлю за кошти Фонду'] = df['Забезпечено закупівлю за кошти Фонду'] * df['Коефіцієнт од. вимірювання']
df['Незабезпечена потреба, абс. знач'] = df['Незабезпечена потреба, абс. знач'] * df['Коефіцієнт од. вимірювання']

df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо df по стовбцях
df = df.groupby(['Вид потреби']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Кабель')

In [ ]:
# -- Кабелі (в розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
#Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення дорівнює
df = df[df['Тип потреби'] == 'Кабельна продукція']
# Залишимо в df лише ті рядки, де в стовбчику 'Одиниця вимірювання' є 'км', або в стовбчику 'Коефіцієнт од. вимірювання' не порожнє значення
df = df[(df['Одиниця вимірювання'] == 'км') | (df['Коефіцієнт од. вимірювання'].notnull())]
# В df залишимо лише ті рядки де в стовбчику "Вид потреби" значення не рівне "Інша кабельна продукція"
df = df[df['Вид потреби'] != 'Інша кабельна продукція']

# В стовбчику 'Коефіцієнт од. вимірювання' всі порожні значення замінимо на 1
df['Коефіцієнт од. вимірювання'].fillna(1, inplace=True)

# Замінимо порожні значення на нуль в таких стовбчиках
df['Загальний обсяг'].fillna(0, inplace=True)
df['Забезпечено'].fillna(0, inplace=True)
df['Забезпечено самостійно, гумдопомога'].fillna(0, inplace=True)
df['Забезпечено самостійно, закупівлі'].fillna(0, inplace=True)
df['Забезпечено закупівлю за кошти Фонду'].fillna(0, inplace=True)
df['Незабезпечена потреба, абс. знач'].fillna(0, inplace=True)
# Значення в стовбчику "Коефіцієнт од. вимірювання"
df['Загальний обсяг'] = df['Загальний обсяг'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено'] = df['Забезпечено'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено самостійно, гумдопомога'] = df['Забезпечено самостійно, гумдопомога'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено самостійно, закупівлі'] = df['Забезпечено самостійно, закупівлі'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено закупівлю за кошти Фонду'] = df['Забезпечено закупівлю за кошти Фонду'] * df['Коефіцієнт од. вимірювання']
df['Незабезпечена потреба, абс. знач'] = df['Незабезпечена потреба, абс. знач'] * df['Коефіцієнт од. вимірювання']

df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише такі стовбці
df = df[['Область', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо df по стовбцях
df = df.groupby(['Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Кабель1')

In [ ]:
# -- Кабелі (в ТОП5 з областями)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
#Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення дорівнює
df = df[df['Тип потреби'] == 'Кабельна продукція']
# Залишимо в df лише ті рядки, де в стовбчику 'Одиниця вимірювання' є 'км', або в стовбчику 'Коефіцієнт од. вимірювання' не порожнє значення
df = df[(df['Одиниця вимірювання'] == 'км') | (df['Коефіцієнт од. вимірювання'].notnull())]
# В df залишимо лише ті рядки де в стовбчику "Вид потреби" значення не рівне "Інша кабельна продукція"
df = df[df['Вид потреби'] != 'Інша кабельна продукція']

# В стовбчику 'Коефіцієнт од. вимірювання' всі порожні значення замінимо на 1
df['Коефіцієнт од. вимірювання'].fillna(1, inplace=True)

# Замінимо порожні значення на нуль в таких стовбчиках
df['Загальний обсяг'].fillna(0, inplace=True)
df['Забезпечено'].fillna(0, inplace=True)
df['Забезпечено самостійно, гумдопомога'].fillna(0, inplace=True)
df['Забезпечено самостійно, закупівлі'].fillna(0, inplace=True)
df['Забезпечено закупівлю за кошти Фонду'].fillna(0, inplace=True)
df['Незабезпечена потреба, абс. знач'].fillna(0, inplace=True)
# Значення в стовбчику "Коефіцієнт од. вимірювання"
df['Загальний обсяг'] = df['Загальний обсяг'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено'] = df['Забезпечено'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено самостійно, гумдопомога'] = df['Забезпечено самостійно, гумдопомога'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено самостійно, закупівлі'] = df['Забезпечено самостійно, закупівлі'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено закупівлю за кошти Фонду'] = df['Забезпечено закупівлю за кошти Фонду'] * df['Коефіцієнт од. вимірювання']
df['Незабезпечена потреба, абс. знач'] = df['Незабезпечена потреба, абс. знач'] * df['Коефіцієнт од. вимірювання']

df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Кабель2')

In [ ]:
# -- Кабелі (в ТОП5 в кого найбільш незадоволені потреби (з областями))
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
#Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення дорівнює
df = df[df['Тип потреби'] == 'Кабельна продукція']
# Залишимо в df лише ті рядки, де в стовбчику 'Одиниця вимірювання' є 'км', або в стовбчику 'Коефіцієнт од. вимірювання' не порожнє значення
df = df[(df['Одиниця вимірювання'] == 'км') | (df['Коефіцієнт од. вимірювання'].notnull())]
# В df залишимо лише ті рядки де в стовбчику "Вид потреби" значення не рівне "Інша кабельна продукція"
df = df[df['Вид потреби'] != 'Інша кабельна продукція']

# В стовбчику 'Коефіцієнт од. вимірювання' всі порожні значення замінимо на 1
df['Коефіцієнт од. вимірювання'].fillna(1, inplace=True)

# Замінимо порожні значення на нуль в таких стовбчиках
df['Загальний обсяг'].fillna(0, inplace=True)
df['Забезпечено'].fillna(0, inplace=True)
df['Забезпечено самостійно, гумдопомога'].fillna(0, inplace=True)
df['Забезпечено самостійно, закупівлі'].fillna(0, inplace=True)
df['Забезпечено закупівлю за кошти Фонду'].fillna(0, inplace=True)
df['Незабезпечена потреба, абс. знач'].fillna(0, inplace=True)
# Значення в стовбчику "Коефіцієнт од. вимірювання"
df['Загальний обсяг'] = df['Загальний обсяг'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено'] = df['Забезпечено'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено самостійно, гумдопомога'] = df['Забезпечено самостійно, гумдопомога'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено самостійно, закупівлі'] = df['Забезпечено самостійно, закупівлі'] * df['Коефіцієнт од. вимірювання']
df['Забезпечено закупівлю за кошти Фонду'] = df['Забезпечено закупівлю за кошти Фонду'] * df['Коефіцієнт од. вимірювання']
df['Незабезпечена потреба, абс. знач'] = df['Незабезпечена потреба, абс. знач'] * df['Коефіцієнт од. вимірювання']

df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Незабезпечена потреба, абс. знач', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Кабель3')

In [ ]:
# Кабелі (ті що відправлені без потреби, в розрізі категорій), потрібні категорії вибираємо вручну
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]

#Залишимо в df лише ті рядки
df = df[df['Тип потреби'] == 'Кабельна продукція']
#Залишимо в df лише ті рядки
df = df[df['Одиниці вимірювання'] == 'км']
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Вид потреби']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Кабелі_відгрузки')

In [ ]:
# Кабелі (ті що відправлені без потреби, в розрізі областей), потрібні категорії вибираємо вручну
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]

#Залишимо в df лише ті рядки
df = df[df['Тип потреби'] == 'Кабельна продукція']
#Залишимо в df лише ті рядки
df = df[df['Одиниці вимірювання'] == 'км']
# Залишимо в df лише такі стовбці
df = df[['Область', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Область']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Кабелі_відгрузки1')

In [ ]:
# Кабелі (пріоритетні потреби, за категорією)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/пріоритетні потреби.xlsx', sheet_name='Лише потрібні категорії', engine='openpyxl')
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[df['Тип потреби'] == 'Кабельна продукція']
#Залишимо в df лише ті рядки
df = df[df['Одиниця вимірювання'] == 'км']
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг 1 пріоритет', 'Загальний обсяг 2 пріоритет']]
# Згрупуємо дані за стовбцями
df = df.groupby('Вид потреби').sum()
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Пріор_кабелі')

In [ ]:
# -- Трансформатори силові (в розрізі категорій)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(["Автотрансформатори", "Трансформатори силові 10-40 кВ", "Трансформатори силові 110-220 кВ", "Трансформатори силові до 6 кВ"])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо df по стовбцях
df = df.groupby(['Вид потреби']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Трансф_силові3')

In [ ]:
# -- Трансформатори силові (в розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(["Автотрансформатори", "Трансформатори силові 10-40 кВ", "Трансформатори силові 110-220 кВ", "Трансформатори силові до 6 кВ"])]
# Залишимо в df лише такі стовбці
df = df[['Область', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо df по стовбцях
df = df.groupby(['Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Трансф_силові4')

In [ ]:
# -- Трансформатори силові (ТОП5 з областями)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(["Автотрансформатори", "Трансформатори силові 10-40 кВ", "Трансформатори силові 110-220 кВ", "Трансформатори силові до 6 кВ"])]
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Трансф_силові5')

In [ ]:
# -- Трансформатори силові (ТОП5 по незабезпечених потребах з областями)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(["Автотрансформатори", "Трансформатори силові 10-40 кВ", "Трансформатори силові 110-220 кВ", "Трансформатори силові до 6 кВ"])]
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Незабезпечена потреба, абс. знач', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Трансф_силові6')

In [ ]:
# Трансформатори силові (ті що відправлені без потреби, в розрізі категорій)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# Залишимо в df лише ті рядки
df = df[df['Вид потреби'].isin(["Автотрансформатори", "Трансформатори силові 10-40 кВ", "Трансформатори силові 110-220 кВ", "Трансформатори силові до 6 кВ"])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Вид потреби']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Трансф_силові_відгрузки')

In [ ]:
# Трансформатори силові (ті що відправлені без потреби, в розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# Залишимо в df лише ті рядки
df = df[df['Вид потреби'].isin(["Автотрансформатори", "Трансформатори силові 10-40 кВ", "Трансформатори силові 110-220 кВ", "Трансформатори силові до 6 кВ"])]
# Залишимо в df лише такі стовбці
df = df[['Область', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Область']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Трансф_силові_відгрузки2')

In [ ]:
# Трансформатори силові (пріоритетні потреби, за потужністю)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/пріоритетні потреби.xlsx', sheet_name='Лише потрібні категорії', engine='openpyxl')
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[df['Вид потреби'].isin(["Автотрансформатори", "Трансформатори силові 10-40 кВ", "Трансформатори силові 110-220 кВ", "Трансформатори силові до 6 кВ"])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг 1 пріоритет', 'Загальний обсяг 2 пріоритет']]
# Згрупуємо дані за стовбцями
df = df.groupby('Вид потреби').sum()
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Пріор_Трвнс_силові')

In [ ]:
# -- Вимірювальні трансформатори (в розрізі категорій)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(["Трансформатори струму та напруги комбіновані (високої напруги)", "Трансформатори струму та напруги комбіновані (середньої напруги)", "Трансформатори напруги 6-35 кВ", "Трансформатори напруги 330 кВ і вище", "Трансформатори напруги 110-220 кВ", "Трансформатори струму (низької напруги)", "Трансформатори струму (середньої напруги)", "Трансформатори струму (високої напруги)"])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо df по стовбцях
df = df.groupby(['Вид потреби']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Трансф_вимір')

In [ ]:
# -- Вимірювальні трансформатори (в розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(["Трансформатори струму та напруги комбіновані (високої напруги)", "Трансформатори струму та напруги комбіновані (середньої напруги)", "Трансформатори напруги 6-35 кВ", "Трансформатори напруги 330 кВ і вище", "Трансформатори напруги 110-220 кВ", "Трансформатори струму (низької напруги)", "Трансформатори струму (середньої напруги)", "Трансформатори струму (високої напруги)"])]
# Залишимо в df лише такі стовбці
df = df[['Область', 'Загальний обсяг', 'Забезпечено загалом', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо df по стовбцях
df = df.groupby(['Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Трансф_вимір1')

In [ ]:
# -- Вимірювальні трансформатори (ТОП5 з областями)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(["Трансформатори струму та напруги комбіновані (високої напруги)", "Трансформатори струму та напруги комбіновані (середньої напруги)", "Трансформатори напруги 6-35 кВ", "Трансформатори напруги 330 кВ і вище", "Трансформатори напруги 110-220 кВ", "Трансформатори струму (низької напруги)", "Трансформатори струму (середньої напруги)", "Трансформатори струму (високої напруги)"])]
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Загальний обсяг', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Трансф_вимір2')

In [ ]:
# -- Вимірювальні трансформатори (ТОП5 невдах з областями)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
df['Забезпечено загалом'] = df['Загальний обсяг'] - df['Незабезпечена потреба, абс. знач']
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(["Трансформатори струму та напруги комбіновані (високої напруги)", "Трансформатори струму та напруги комбіновані (середньої напруги)", "Трансформатори напруги 6-35 кВ", "Трансформатори напруги 330 кВ і вище", "Трансформатори напруги 110-220 кВ", "Трансформатори струму (низької напруги)", "Трансформатори струму (середньої напруги)", "Трансформатори струму (високої напруги)"])]
# Залишимо в df лише такі стовбці
df = df[['Назва  Набувача', 'Область', 'Загальний обсяг', 'Незабезпечена потреба, абс. знач']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Назва  Набувача', 'Область']).sum()
# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Незабезпечена потреба, абс. знач', ascending=False)
# Залишимо лише 5 перших рядків
df = df.head(5)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Трансф_вимір3')

In [ ]:
# Трансформатори вимірювальні (ті що відправлені без потреби, в розрізі категорій)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# Залишимо в df лише ті рядки
df = df[df['Вид потреби'].isin(["Трансформатори струму та напруги комбіновані (високої напруги)", "Трансформатори струму та напруги комбіновані (середньої напруги)", "Трансформатори напруги 6-35 кВ", "Трансформатори напруги 330 кВ і вище", "Трансформатори напруги 110-220 кВ", "Трансформатори струму (низької напруги)", "Трансформатори струму (середньої напруги)", "Трансформатори струму (високої напруги)"])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Вид потреби']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Трансф_вимір_відгрузки')

In [ ]:
# Трансформатори вимірювальні (ті що відправлені без потреби, в розрізі областей)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl')
# В df залишимо лише ті рядки в яких в стовбчику "ID Позиції" значення або "PT100001" або порожнє значення
df = df[(df['ID Позиції'] == 'PT100001') | (df['ID Позиції'].isnull())]
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# Залишимо в df лише ті рядки
df = df[df['Вид потреби'].isin(["Трансформатори струму та напруги комбіновані (високої напруги)", "Трансформатори струму та напруги комбіновані (середньої напруги)", "Трансформатори напруги 6-35 кВ", "Трансформатори напруги 330 кВ і вище", "Трансформатори напруги 110-220 кВ", "Трансформатори струму (низької напруги)", "Трансформатори струму (середньої напруги)", "Трансформатори струму (високої напруги)"])]
# Залишимо в df лише такі стовбці
df = df[['Область', 'Обсяг']]
# Згрупуємо дані за стовбцем
df = df.groupby(['Область']).sum()

# Відсортуємо df по стовбцю
# df = df.sort_values(by='Обсяг', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Трансф_вимір_відгрузки1')

In [ ]:
# Трансформатори вимірювальні (пріоритетні потреби, за потужністю)
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/пріоритетні потреби.xlsx', sheet_name='Лише потрібні категорії', engine='openpyxl')
# Залишимо в df лише ті рядки, де в стовбці "Тип потреби" значення рівне "Електростанції (генератори) та обладнання", і в стовбчику "Вид потреби" значення рівне "Газотурбінні установки"
df = df[df['Вид потреби'].isin(["Трансформатори струму та напруги комбіновані (високої напруги)", "Трансформатори струму та напруги комбіновані (середньої напруги)", "Трансформатори напруги 6-35 кВ", "Трансформатори напруги 330 кВ і вище", "Трансформатори напруги 110-220 кВ", "Трансформатори струму (низької напруги)", "Трансформатори струму (середньої напруги)", "Трансформатори струму (високої напруги)"])]
# Залишимо в df лише такі стовбці
df = df[['Вид потреби', 'Загальний обсяг 1 пріоритет', 'Загальний обсяг 2 пріоритет']]
# Згрупуємо дані за стовбцями
df = df.groupby('Вид потреби').sum()
# Додамо до вже існуючого файлу result.xlsx вкладку із даними df (ігноруючи індекси)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Пріор_Трвнс_вимір')

In [ ]:
# -- ТОП категорії
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
df = df[['Тип потреби']]
# Додамо стовбчик "Кількість"
df['Кількість'] = 1
# Згрупуємо по типу потреби
df = df.groupby('Тип потреби').sum()
# Відсортуємо df по стовбцю
df = df.sort_values(by='Кількість', ascending=False)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='TOP cat0')

In [ ]:
# -- ТОП категорії в розрізі сфер
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]
# #Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не порожнє
# df = df[df['Зона бойових дій'].notna()]
# Залишимо в df лише ті рядки, де в стовбці "Вид потреби" значення рівні такому:
df = df[df['Вид потреби'].isin(["-------"])]

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid_new.xlsx', sheet_name="Відгрузки", engine='openpyxl', converters={'ЄДРПОУ': str, 'ЄДРПОУ філії': str})
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='from_aid_new_Відгрузки')

In [ ]:
# Беремо з вкладок "Потреби" і "Набувачі" та переносимо у вкладку "Відгрузки" такі стовбчики: "Тип потреби"	"Вид потреби	Сфера діяльності"
# "Вид діяльності"	"Область"	"Зона бойових дій"	"Власність"	"Одиниця вимірювання"	"Загальний обсяг" "Забезпечено".
# Для відгрузок з ID Позиції порожніми або PT100001 Тип потреби і Вид потреби робимо вручну, або якось по іншому
df_aid = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Відгрузки", engine='openpyxl', converters={'ЄДРПОУ': str, 'ЄДРПОУ філії': str})
df_aid_potreby = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Потреби", engine='openpyxl', converters={'ЄДРПОУ  Набувача': str, 'ЄДРПОУ філії': str})
df_aid_nabuvachi = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Набувачі", engine='openpyxl', converters={'ЄДРПОУ  Набувача': str, 'ЄДРПОУ': str})

# Порівняємо df_aid_potreby і df_aid по стовбцю "ID Позиції" і якщо вони співпадають, то в df_aid в стовбчик "Тип потреби" внесемо значення з df_vidgruzky в стовбчик "Тип потреби"
df_aid.loc[df_aid['ID Позиції'].isin(df_aid_potreby['ID Позиції']), 'Тип потреби1'] = df_aid_potreby['Тип потреби']
df_aid.loc[df_aid['ID Позиції'].isin(df_aid_potreby['ID Позиції']), 'Вид потреби1'] = df_aid_potreby['Вид потреби']
#Сфера діяльності
df_aid.loc[df_aid['ЄДРПОУ'].isin(df_aid_nabuvachi['ЄДРПОУ']), 'Сфера діяльності'] = df_aid_nabuvachi['Сфера діяльності']
#Вид діяльності
df_aid.loc[df_aid['ЄДРПОУ'].isin(df_aid_nabuvachi['ЄДРПОУ']), 'Вид діяльності'] = df_aid_nabuvachi['Вид діяльності']
#Область
df_aid.loc[df_aid['ЄДРПОУ'].isin(df_aid_nabuvachi['ЄДРПОУ']), 'Область'] = df_aid_nabuvachi['Область']
#Зона бойових дій
df_aid.loc[df_aid['ЄДРПОУ'].isin(df_aid_nabuvachi['ЄДРПОУ']), 'Зона бойових дій'] = df_aid_nabuvachi['Зона бойових дій']
#Власність
df_aid.loc[df_aid['ЄДРПОУ'].isin(df_aid_nabuvachi['ЄДРПОУ']), 'Власність'] = df_aid_nabuvachi['Власність']
#Одиниця вимірювання
df_aid.loc[df_aid['ID Позиції'].isin(df_aid_potreby['ID Позиції']), 'Одиниця вимірювання'] = df_aid_potreby['Одиниця вимірювання']
#Загальний обсяг
df_aid.loc[df_aid['ID Позиції'].isin(df_aid_potreby['ID Позиції']), 'Загальний обсяг'] = df_aid_potreby['Загальний обсяг']
#Забезпечено
df_aid.loc[df_aid['ID Позиції'].isin(df_aid_potreby['ID Позиції']), 'Забезпечено'] = df_aid_potreby['Забезпечено']
# Якщо в df_aid в стовбчику 'Тип потреби' значення порожнє, то вставляємо  туди значення з стовбчика 'Тип потреби1'
df_aid['Тип потреби'] = df_aid['Тип потреби'].fillna(df_aid['Тип потреби1'])
df_aid['Вид потреби'] = df_aid['Вид потреби'].fillna(df_aid['Вид потреби1'])
# Видаляємо стовбчики 'Тип потреби1' та 'Вид потреби1'
df_aid = df_aid.drop(['Тип потреби1', 'Вид потреби1'], axis=1)


# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', mode='a') as writer:
    df_aid.to_excel(writer, sheet_name='Відгрузки2')

In [ ]:
# -- Свіжоскачаний файл energoAid просто перепишемо в файл temp.xlsx додавши до листа "Потреби"
# з листа "Набувачі" стовбчики "Сфера діяльності", "Вид діяльності", "Область", "Зона бойових дій", "Власність"
# Відкриємо  файл (всі листи). Стовбчики ""ЄДРПОУ  Набувача"", "ЄДРПОУ" та "ЄДРПОУ філії" конвертуємо в текст
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/Energo-Aid 0.5.xlsx', sheet_name=None, engine='openpyxl', converters={'ЄДРПОУ  Набувача': str, 'ЄДРПОУ філії': str, 'ЄДРПОУ': str})
# # В листі потреби перейменуємо Заголовок стовбчика "ЄДРПОУ  Набувача" на "ЄДРПОУ"
# df['Потреби'].rename(columns={'ЄДРПОУ  Набувача': 'ЄДРПОУ'}, inplace=True)
# # В стовбчик "Сфера діяльності" (лист "Потреби") помістимо значення з стовбчика "Сфера діяльності" (лист "Набувачі") (коли в них співпадає значення стовбчика "ЄДРПОУ)
# df['Потреби']['Сфера діяльності'] = df['Потреби'].apply(lambda row: df['Набувачі'][df['Набувачі']['ЄДРПОУ'] == row['ЄДРПОУ']]['Сфера діяльності'].values[0] if row['ЄДРПОУ'] in df['Набувачі']['ЄДРПОУ'].values else '', axis=1)
# # Те саме тільки для стовбчика "Вид діяльності" (замість "Сфера діяльності" вище)
# df['Потреби']['Вид діяльності'] = df['Потреби'].apply(lambda row: df['Набувачі'][df['Набувачі']['ЄДРПОУ'] == row['ЄДРПОУ']]['Вид діяльності'].values[0] if row['ЄДРПОУ'] in df['Набувачі']['ЄДРПОУ'].values else '', axis=1)
# # Те саме тільки для стовбчика "Область"
# df['Потреби']['Область'] = df['Потреби'].apply(lambda row: df['Набувачі'][df['Набувачі']['ЄДРПОУ'] == row['ЄДРПОУ']]['Область'].values[0] if row['ЄДРПОУ'] in df['Набувачі']['ЄДРПОУ'].values else '', axis=1)
# # Те саме тільки для стовбчика "Зона бойових дій"
# df['Потреби']['Зона бойових дій'] = df['Потреби'].apply(lambda row: df['Набувачі'][df['Набувачі']['ЄДРПОУ'] == row['ЄДРПОУ']]['Зона бойових дій'].values[0] if row['ЄДРПОУ'] in df['Набувачі']['ЄДРПОУ'].values else '', axis=1)
# # Те саме тільки для стовбчика "Власність"
# df['Потреби']['Власність'] = df['Потреби'].apply(lambda row: df['Набувачі'][df['Набувачі']['ЄДРПОУ'] == row['ЄДРПОУ']]['Власність'].values[0] if row['ЄДРПОУ'] in df['Набувачі']['ЄДРПОУ'].values else '', axis=1)
# # В листі потреби перейменуємо Заголовок стовбчика "ЄДРПОУ" на "ЄДРПОУ  Набувача"
# df['Потреби'].rename(columns={'ЄДРПОУ': 'ЄДРПОУ  Набувача'}, inplace=True)
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx') as writer:
    for sheet_name, df_sheet in df.items():
        df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
# Найважливіші види запитів Підрахунок кількості запитів в розрізі областей і задоволення потреб
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Потреби", engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]

df = df[df['Вид потреби'].isin(["Засоби телекомунікації та телемеханіки", "Комп'ютери та ноутбуки", "Автовишки (автопiдйомники з люлькою)", "Автокрани", "Автомобілі бригадні вантажо-пасажирські", "Автомобілі для перевезення залізобетонних опор (опоровози)", "Автомобілі з маніпулятором", "Автомобілі самоскиди", "Автомобілі тягачі", "Автонавантажувачі", "Автопідйомники", "Автотранспорт (інше)", "Бурильно-кранові машини", "Екскаватори", "Електротехнічні лабораторії", "Трактори", "Причіпи автомобільні", "Спецтехніка (інше)", "Арматура запірна", "Арматура інша", "Арматура лінійна", "Арматура СІП", "Арматура нафтогазова", "Затискачі електричні з'єднувальні і натяжні (лінійні та апаратні)", "Муфти", "Наконечники алюмінієві", "Наконечники мідні", "Термічні клапани", "Траверси", "Вводи вимикачів", "Вимикачі автоматичні", "Вимикачі вакуумні", "Вимикачі елегазові", "Вимикачі повітрянні", "Вимикачі оливні", "Інше обладнання для вимикачів", "Камери дугогасіння", "Обладнання для елегазових вимикачів", "Перемикачі", "Роз'єднувачі", "Реклоузери", "Рубильники", "Гідравлічні двигуни", "Електродвигуни < 10 кВт", "Електродвигуни 11-110 кВт", "Електродвигуни > 110 кВт", "Двигуни (інше)", "Акумуляторні батареї", "Джерела безперебійного живлення", "<10 кВт", "10-30 кВт", "30-100 кВт", "100-300 кВт", "300-500 кВт", "500-700 кВт", "700-1000 кВт", "1000-1500 кВт", "1500-2000 кВт", ">2000 кВт", "Генератори (запчастини, сервісні, ремонтні комплекти, інше)", "Запчастини", "Інвертори", "Інші засоби та обладнання", "Котельне обладнання", "Машини реверсивні", "Металошукачі", "Обладнання для вітрових електростанцій", "Обладнання для газових мереж", "Обладнання для сонячних електростанцій", "Обладнання для шахт", "Освітлювальні пристрої", "Регулятор тиску газу", "Трасошукачі газові", "Ізолятори", "Ізолятори підвісні", "Інша кабельна продукція", "Кабель з'єднувальний", "Кабель контрольний", "Кабель силовий < 0,4 кВ", "Кабель силовий 0,4-1 кВ", "Кабель силовий 6-35 кВ", "Кабель силовий > 110 кВ", "Провід ізольований", "Провід неізольований", "Провід самонесучий ізольований (СІП)", "Компресори (інші)", "Компресори підстанційні", "Інші матеріали", "Матеріали для ремонту газорозподільчої мережі", "Матеріали для ремонту електромережі", "Матеріали зі сталі", "Заземлювачі нейтралі", "Комплекти тимчасасового заземлення", "Матеріали з дерева", "Провід", "Стрічка конвеєрна", "Трос грозозахисний", "Троси металеві", "Шини електричні", "Домкрати", "Інші механізми", "Машини для натяжки проводу та тросу", "Механізми спеціальні", "Насос різного призначення", "Преси гідравлічні", "Високочастотні обмежувачі", "Обмежувачі перенапруг", "Розрядники", "Оливи автомобільні", "Оливи інші", "Оливи трансформаторні", "Обладнання до опор", "Опори дерев'янні", "Опори залізобетонні", "Опори інші", "Опори металеві", "Бензин", "Газ", "Дизель", "Паливо для електростанцій", "Вимірювальні прилади", "Інші прилади обліку", "Лічильники", "Газорегуляторні пункти", "Реактори", "Комплектний розподільний пристрій (КРП)", "Інше обладнання релейного захисту та автоматики", "Автотрансформатори", "Вводи трансформаторні", "Інше трансформаторне обладнання", "Мобільні підстанції", "Трансформатори блочні", "Трансформатори напруги 6-35 кВ", "Трансформатори напруги 110-220 кВ", "Трансформатори напруги 330 кВ і вище", "Трансформаторні підстанції середньої напруги", "Трансформаторні підстанції високої напруги", "Трансформатори регулюючі", "Трансформатори силові до 6 кВ", "Трансформатори силові 10-40 кВ", "Трансформатори силові 110-220 кВ", "Трансформатори силові 330 кВ і вище", "Трансформатори струму (низької напруги)", "Трансформатори струму (середньої напруги)", "Трансформатори струму (високої напруги)", "Трансформатори струму та напруги комбіновані (середньої напруги)", "Трансформатори струму та напруги комбіновані (високої напруги)", "Труби для прокладання кабелю", "Трубна продукція", "Газотурбінні установки", "Парові турбіни", "Турбіни", "Інше турбогенераторне обладнання", "Блочний щит керування", "Мобільні приміщення", "Шафи та ящики", "Щити керування"])]

# Якщо в стовбчику "Виконання, %" значення >100, то замінимо його на 100
df.loc[df['Виконання, %'] > 100, 'Виконання, %'] = 100
# Додамо стовбчик "Кількість потреб" з значеннями 1
df['Кількість потреб'] = 1


####---------
# В датафрейм df_two запишемо рядки з df, де значення стовбчика "Назва  Набувача" дорівнює 'ПрАТ "ДОНЕЦЬКОБЛГАЗ"' або 'ПрАТ "ТЕРНОПІЛЬГАЗ"'
df_two = df[(df['Назва  Набувача'] == 'ПрАТ "ДОНЕЦЬКОБЛГАЗ"') | (df['Назва  Набувача'] == 'ПрАТ "ТЕРНОПІЛЬГАЗ"')]
# В датафреймі df видалимо рядки, де значення стовбчика "Назва  Набувача" дорівнює 'ПрАТ "ДОНЕЦЬКОБЛГАЗ"' або 'ПрАТ "ТЕРНОПІЛЬГАЗ"'
df = df.drop(df[(df['Назва  Набувача'] == 'ПрАТ "ДОНЕЦЬКОБЛГАЗ"') | (df['Назва  Набувача'] == 'ПрАТ "ТЕРНОПІЛЬГАЗ"')].index)

# Залишимо такі стовбчики
df = df[['Область', 'Кількість потреб', 'Виконання, %']]
df_two = df_two[['Повне найменування потреби', 'Область', 'Кількість потреб', 'Виконання, %']]

# Згрупуємо по областях ('Кількість потреб', і 'Виконання, %' візьмемо середнє значення, це прибере однакові назви товарів, але при цьому кількість потреб не зміниться, бо воно не сумуалось)
df_two = df_two.groupby(['Область', 'Повне найменування потреби']).agg({'Кількість потреб': 'mean', 'Виконання, %': 'mean'}).reset_index()
# Залишимо такі стовбчики
df_two = df_two[['Область', 'Кількість потреб', 'Виконання, %']]
df_two = df_two.groupby('Область').agg({'Кількість потреб': 'sum', 'Виконання, %': 'mean'}).reset_index()
# Згрупуємо по областях (підсумуємо 'Кількість потреб', а 'Виконання, %' візьмемо середнє значення)
df = df.groupby('Область').agg({'Кількість потреб': 'sum', 'Виконання, %': 'mean'}).reset_index()
# об'єднаємо df_two і df
df = pd.concat([df, df_two], ignore_index=True)
####-----


# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Кількість потреб', ascending=False)
# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='ТОП потреби2')

In [ ]:
# Найважливіші види запитів Підрахунок кількості запитів в розрізі сфер діяльності і задоволення потреб
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Потреби", engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]

df = df[df['Вид потреби'].isin(["Засоби телекомунікації та телемеханіки", "Комп'ютери та ноутбуки", "Автовишки (автопiдйомники з люлькою)", "Автокрани", "Автомобілі бригадні вантажо-пасажирські", "Автомобілі для перевезення залізобетонних опор (опоровози)", "Автомобілі з маніпулятором", "Автомобілі самоскиди", "Автомобілі тягачі", "Автонавантажувачі", "Автопідйомники", "Автотранспорт (інше)", "Бурильно-кранові машини", "Екскаватори", "Електротехнічні лабораторії", "Трактори", "Причіпи автомобільні", "Спецтехніка (інше)", "Арматура запірна", "Арматура інша", "Арматура лінійна", "Арматура СІП", "Арматура нафтогазова", "Затискачі електричні з'єднувальні і натяжні (лінійні та апаратні)", "Муфти", "Наконечники алюмінієві", "Наконечники мідні", "Термічні клапани", "Траверси", "Вводи вимикачів", "Вимикачі автоматичні", "Вимикачі вакуумні", "Вимикачі елегазові", "Вимикачі повітрянні", "Вимикачі оливні", "Інше обладнання для вимикачів", "Камери дугогасіння", "Обладнання для елегазових вимикачів", "Перемикачі", "Роз'єднувачі", "Реклоузери", "Рубильники", "Гідравлічні двигуни", "Електродвигуни < 10 кВт", "Електродвигуни 11-110 кВт", "Електродвигуни > 110 кВт", "Двигуни (інше)", "Акумуляторні батареї", "Джерела безперебійного живлення", "<10 кВт", "10-30 кВт", "30-100 кВт", "100-300 кВт", "300-500 кВт", "500-700 кВт", "700-1000 кВт", "1000-1500 кВт", "1500-2000 кВт", ">2000 кВт", "Генератори (запчастини, сервісні, ремонтні комплекти, інше)", "Запчастини", "Інвертори", "Інші засоби та обладнання", "Котельне обладнання", "Машини реверсивні", "Металошукачі", "Обладнання для вітрових електростанцій", "Обладнання для газових мереж", "Обладнання для сонячних електростанцій", "Обладнання для шахт", "Освітлювальні пристрої", "Регулятор тиску газу", "Трасошукачі газові", "Ізолятори", "Ізолятори підвісні", "Інша кабельна продукція", "Кабель з'єднувальний", "Кабель контрольний", "Кабель силовий < 0,4 кВ", "Кабель силовий 0,4-1 кВ", "Кабель силовий 6-35 кВ", "Кабель силовий > 110 кВ", "Провід ізольований", "Провід неізольований", "Провід самонесучий ізольований (СІП)", "Компресори (інші)", "Компресори підстанційні", "Інші матеріали", "Матеріали для ремонту газорозподільчої мережі", "Матеріали для ремонту електромережі", "Матеріали зі сталі", "Заземлювачі нейтралі", "Комплекти тимчасасового заземлення", "Матеріали з дерева", "Провід", "Стрічка конвеєрна", "Трос грозозахисний", "Троси металеві", "Шини електричні", "Домкрати", "Інші механізми", "Машини для натяжки проводу та тросу", "Механізми спеціальні", "Насос різного призначення", "Преси гідравлічні", "Високочастотні обмежувачі", "Обмежувачі перенапруг", "Розрядники", "Оливи автомобільні", "Оливи інші", "Оливи трансформаторні", "Обладнання до опор", "Опори дерев'янні", "Опори залізобетонні", "Опори інші", "Опори металеві", "Бензин", "Газ", "Дизель", "Паливо для електростанцій", "Вимірювальні прилади", "Інші прилади обліку", "Лічильники", "Газорегуляторні пункти", "Реактори", "Комплектний розподільний пристрій (КРП)", "Інше обладнання релейного захисту та автоматики", "Автотрансформатори", "Вводи трансформаторні", "Інше трансформаторне обладнання", "Мобільні підстанції", "Трансформатори блочні", "Трансформатори напруги 6-35 кВ", "Трансформатори напруги 110-220 кВ", "Трансформатори напруги 330 кВ і вище", "Трансформаторні підстанції середньої напруги", "Трансформаторні підстанції високої напруги", "Трансформатори регулюючі", "Трансформатори силові до 6 кВ", "Трансформатори силові 10-40 кВ", "Трансформатори силові 110-220 кВ", "Трансформатори силові 330 кВ і вище", "Трансформатори струму (низької напруги)", "Трансформатори струму (середньої напруги)", "Трансформатори струму (високої напруги)", "Трансформатори струму та напруги комбіновані (середньої напруги)", "Трансформатори струму та напруги комбіновані (високої напруги)", "Труби для прокладання кабелю", "Трубна продукція", "Газотурбінні установки", "Парові турбіни", "Турбіни", "Інше турбогенераторне обладнання", "Блочний щит керування", "Мобільні приміщення", "Шафи та ящики", "Щити керування"])]

# Якщо в стовбчику "Виконання, %" значення >100, то замінимо його на 100
df.loc[df['Виконання, %'] > 100, 'Виконання, %'] = 100

# Додамо стовбчик "Кількість потреб" з значеннями 1
df['Кількість потреб'] = 1

####---------
# В датафрейм df_two запишемо рядки з df, де значення стовбчика "Назва  Набувача" дорівнює 'ПрАТ "ДОНЕЦЬКОБЛГАЗ"' або 'ПрАТ "ТЕРНОПІЛЬГАЗ"'
df_two = df[(df['Назва  Набувача'] == 'ПрАТ "ДОНЕЦЬКОБЛГАЗ"') | (df['Назва  Набувача'] == 'ПрАТ "ТЕРНОПІЛЬГАЗ"')]
# В датафреймі df видалимо рядки, де значення стовбчика "Назва  Набувача" дорівнює 'ПрАТ "ДОНЕЦЬКОБЛГАЗ"' або 'ПрАТ "ТЕРНОПІЛЬГАЗ"'
df = df.drop(df[(df['Назва  Набувача'] == 'ПрАТ "ДОНЕЦЬКОБЛГАЗ"') | (df['Назва  Набувача'] == 'ПрАТ "ТЕРНОПІЛЬГАЗ"')].index)

# Залишимо такі стовбчики
df = df[['Сфера діяльності', 'Кількість потреб', 'Виконання, %']]
df_two = df_two[['Повне найменування потреби', 'Сфера діяльності', 'Кількість потреб', 'Виконання, %']]

# Згрупуємо по областях ('Кількість потреб', і 'Виконання, %' візьмемо середнє значення, це прибере однакові назви товарів, але при цьому кількість потреб не зміниться, бо воно не сумуалось)
df_two = df_two.groupby(['Сфера діяльності', 'Повне найменування потреби']).agg({'Кількість потреб': 'mean', 'Виконання, %': 'mean'}).reset_index()
# Залишимо такі стовбчики
df_two = df_two[['Сфера діяльності', 'Кількість потреб', 'Виконання, %']]
df_two = df_two.groupby('Сфера діяльності').agg({'Кількість потреб': 'sum', 'Виконання, %': 'mean'}).reset_index()
# Згрупуємо по областях (підсумуємо 'Кількість потреб', а 'Виконання, %' візьмемо середнє значення)
df = df.groupby('Сфера діяльності').agg({'Кількість потреб': 'sum', 'Виконання, %': 'mean'}).reset_index()
# об'єднаємо df_two і df
df = pd.concat([df, df_two], ignore_index=True)
####-----

# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Кількість потреб', ascending=False)

# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='ТОП потреби3')

In [ ]:
# Найважливіші види запитів Підрахунок кількості запитів в розрізі Назви Набувача і задоволення потреб
df = pd.read_excel('/content/drive/MyDrive/energy-map/public/aid/aid.xlsx', sheet_name="Потреби", engine='openpyxl')
#Залишимо в df лише ті рядки, де в стовбці "Зона бойових дій" значення не дорівнює "-1"
df = df[df['Зона бойових дій'] != -1]

df = df[df['Вид потреби'].isin(["Засоби телекомунікації та телемеханіки", "Комп'ютери та ноутбуки", "Автовишки (автопiдйомники з люлькою)", "Автокрани", "Автомобілі бригадні вантажо-пасажирські", "Автомобілі для перевезення залізобетонних опор (опоровози)", "Автомобілі з маніпулятором", "Автомобілі самоскиди", "Автомобілі тягачі", "Автонавантажувачі", "Автопідйомники", "Автотранспорт (інше)", "Бурильно-кранові машини", "Екскаватори", "Електротехнічні лабораторії", "Трактори", "Причіпи автомобільні", "Спецтехніка (інше)", "Арматура запірна", "Арматура інша", "Арматура лінійна", "Арматура СІП", "Арматура нафтогазова", "Затискачі електричні з'єднувальні і натяжні (лінійні та апаратні)", "Муфти", "Наконечники алюмінієві", "Наконечники мідні", "Термічні клапани", "Траверси", "Вводи вимикачів", "Вимикачі автоматичні", "Вимикачі вакуумні", "Вимикачі елегазові", "Вимикачі повітрянні", "Вимикачі оливні", "Інше обладнання для вимикачів", "Камери дугогасіння", "Обладнання для елегазових вимикачів", "Перемикачі", "Роз'єднувачі", "Реклоузери", "Рубильники", "Гідравлічні двигуни", "Електродвигуни < 10 кВт", "Електродвигуни 11-110 кВт", "Електродвигуни > 110 кВт", "Двигуни (інше)", "Акумуляторні батареї", "Джерела безперебійного живлення", "<10 кВт", "10-30 кВт", "30-100 кВт", "100-300 кВт", "300-500 кВт", "500-700 кВт", "700-1000 кВт", "1000-1500 кВт", "1500-2000 кВт", ">2000 кВт", "Генератори (запчастини, сервісні, ремонтні комплекти, інше)", "Запчастини", "Інвертори", "Інші засоби та обладнання", "Котельне обладнання", "Машини реверсивні", "Металошукачі", "Обладнання для вітрових електростанцій", "Обладнання для газових мереж", "Обладнання для сонячних електростанцій", "Обладнання для шахт", "Освітлювальні пристрої", "Регулятор тиску газу", "Трасошукачі газові", "Ізолятори", "Ізолятори підвісні", "Інша кабельна продукція", "Кабель з'єднувальний", "Кабель контрольний", "Кабель силовий < 0,4 кВ", "Кабель силовий 0,4-1 кВ", "Кабель силовий 6-35 кВ", "Кабель силовий > 110 кВ", "Провід ізольований", "Провід неізольований", "Провід самонесучий ізольований (СІП)", "Компресори (інші)", "Компресори підстанційні", "Інші матеріали", "Матеріали для ремонту газорозподільчої мережі", "Матеріали для ремонту електромережі", "Матеріали зі сталі", "Заземлювачі нейтралі", "Комплекти тимчасасового заземлення", "Матеріали з дерева", "Провід", "Стрічка конвеєрна", "Трос грозозахисний", "Троси металеві", "Шини електричні", "Домкрати", "Інші механізми", "Машини для натяжки проводу та тросу", "Механізми спеціальні", "Насос різного призначення", "Преси гідравлічні", "Високочастотні обмежувачі", "Обмежувачі перенапруг", "Розрядники", "Оливи автомобільні", "Оливи інші", "Оливи трансформаторні", "Обладнання до опор", "Опори дерев'янні", "Опори залізобетонні", "Опори інші", "Опори металеві", "Бензин", "Газ", "Дизель", "Паливо для електростанцій", "Вимірювальні прилади", "Інші прилади обліку", "Лічильники", "Газорегуляторні пункти", "Реактори", "Комплектний розподільний пристрій (КРП)", "Інше обладнання релейного захисту та автоматики", "Автотрансформатори", "Вводи трансформаторні", "Інше трансформаторне обладнання", "Мобільні підстанції", "Трансформатори блочні", "Трансформатори напруги 6-35 кВ", "Трансформатори напруги 110-220 кВ", "Трансформатори напруги 330 кВ і вище", "Трансформаторні підстанції середньої напруги", "Трансформаторні підстанції високої напруги", "Трансформатори регулюючі", "Трансформатори силові до 6 кВ", "Трансформатори силові 10-40 кВ", "Трансформатори силові 110-220 кВ", "Трансформатори силові 330 кВ і вище", "Трансформатори струму (низької напруги)", "Трансформатори струму (середньої напруги)", "Трансформатори струму (високої напруги)", "Трансформатори струму та напруги комбіновані (середньої напруги)", "Трансформатори струму та напруги комбіновані (високої напруги)", "Труби для прокладання кабелю", "Трубна продукція", "Газотурбінні установки", "Парові турбіни", "Турбіни", "Інше турбогенераторне обладнання", "Блочний щит керування", "Мобільні приміщення", "Шафи та ящики", "Щити керування"])]

# Якщо в стовбчику "Виконання, %" значення >100, то замінимо його на 100
df.loc[df['Виконання, %'] > 100, 'Виконання, %'] = 100

# Додамо стовбчик "Кількість потреб" з значеннями 1
df['Кількість потреб'] = 1

####---------
# В датафрейм df_two запишемо рядки з df, де значення стовбчика "Назва  Набувача" дорівнює 'ПрАТ "ДОНЕЦЬКОБЛГАЗ"' або 'ПрАТ "ТЕРНОПІЛЬГАЗ"'
df_two = df[(df['Назва  Набувача'] == 'ПрАТ "ДОНЕЦЬКОБЛГАЗ"') | (df['Назва  Набувача'] == 'ПрАТ "ТЕРНОПІЛЬГАЗ"')]
# В датафреймі df видалимо рядки, де значення стовбчика "Назва  Набувача" дорівнює 'ПрАТ "ДОНЕЦЬКОБЛГАЗ"' або 'ПрАТ "ТЕРНОПІЛЬГАЗ"'
df = df.drop(df[(df['Назва  Набувача'] == 'ПрАТ "ДОНЕЦЬКОБЛГАЗ"') | (df['Назва  Набувача'] == 'ПрАТ "ТЕРНОПІЛЬГАЗ"')].index)

# Залишимо такі стовбчики
df = df[['Назва  Набувача', 'Кількість потреб', 'Виконання, %']]
df_two = df_two[['Повне найменування потреби', 'Назва  Набувача', 'Кількість потреб', 'Виконання, %']]

# Згрупуємо по областях ('Кількість потреб', і 'Виконання, %' візьмемо середнє значення, це прибере однакові назви товарів, але при цьому кількість потреб не зміниться, бо воно не сумуалось)
df_two = df_two.groupby(['Назва  Набувача', 'Повне найменування потреби']).agg({'Кількість потреб': 'mean', 'Виконання, %': 'mean'}).reset_index()
# Залишимо такі стовбчики
df_two = df_two[['Назва  Набувача', 'Кількість потреб', 'Виконання, %']]
df_two = df_two.groupby('Назва  Набувача').agg({'Кількість потреб': 'sum', 'Виконання, %': 'mean'}).reset_index()
# Згрупуємо по областях (підсумуємо 'Кількість потреб', а 'Виконання, %' візьмемо середнє значення)
df = df.groupby('Назва  Набувача').agg({'Кількість потреб': 'sum', 'Виконання, %': 'mean'}).reset_index()
# об'єднаємо df_two і df
df = pd.concat([df, df_two], ignore_index=True)
####-----

# Відсортуємо df по стовбцю 'Загальний обсяг' (від найбільшого до найменшого)
df = df.sort_values(by='Кількість потреб', ascending=False)
# Залишимо лише 10 перших рядків
# df = df.head(10)


# Додамо до вже існуючого файлу
with pd.ExcelWriter('/content/drive/MyDrive/energy-map/public/aid/temp.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='ТОП потреби4')